In [1]:
#%pip3 install opencv-python -q
#%pip3 install mediapipe -q 
#%pip3 install matplotlib
#%pip3 install numpy
#%pip3 install p

import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np 

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose 



In [2]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract Landmarks 
        try: 
            landmarks = results.pose_landmarks.landmark 
            print(landmarks)
        except: 
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    


I0000 00:00:1702623442.975052       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-20.6.4), renderer: Intel(R) UHD Graphics 630
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[x: 0.76562333
y: 0.6583554
z: -0.10983973
visibility: 0.8276306
, x: 0.7753752
y: 0.64449376
z: -0.11256655
visibility: 0.86319613
, x: 0.7781874
y: 0.6465189
z: -0.11268075
visibility: 0.8286092
, x: 0.7809782
y: 0.64861584
z: -0.11287175
visibility: 0.9062559
, x: 0.7675797
y: 0.63610023
z: -0.11969292
visibility: 0.85977674
, x: 0.76479983
y: 0.63200384
z: -0.11975091
visibility: 0.7820255
, x: 0.76203054
y: 0.6279207
z: -0.11990268
visibility: 0.8297387
, x: 0.78312343
y: 0.6456803
z: -0.08627424
visibility: 0.9062855
, x: 0.75579226
y: 0.6152783
z: -0.117219165
visibility: 0.72439814
, x: 0.7644719
y: 0.6699088
z: -0.09398762
visibility: 0.91384786
, x: 0.75554484
y: 0.6597878
z: -0.103105396
visibility: 0.87220114
, x: 0.7899809
y: 0.71271145
z: -0.06891582
visibility: 0.9987245
, x: 0.70704216
y: 0.6147474
z: -0.095521994
visibility: 0.99285567
, x: 0.7997244
y: 0.8383986
z: -0.07427437
visibility: 0.5586631
, x: 0.6586775
y: 0.6768587
z: -0.14190105
visibility: 0.18247128
, x:

In [3]:
mp_drawing.DrawingSpec??

Init signature:
mp_drawing.DrawingSpec(
    color: Tuple[int, int, int] = (224, 224, 224),
    thickness: int = 2,
    circle_radius: int = 2,
) -> None
Docstring:      DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
Source:        
@dataclasses.dataclass
class DrawingSpec:
  # Color for drawing the annotation. Default to the white color.
  color: Tuple[int, int, int] = WHITE_COLOR
  # Thickness for drawing the annotation. Default to 2 pixels.
  thickness: int = 2
  # Circle radius. Default to 2 pixels.
  circle_radius: int = 2
File:           /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mediapipe/python/solutions/drawing_utils.py
Type:           type
Subclasses:     